In [ ]:
import tensorflow as tf
tf.random.set_seed(1)
tf.compat.v1.enable_eager_execution()

import numpy as np
np.random.seed(1)

import gensim
import pandas as pd
import matplotlib.pyplot as plt
import talos as ta

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, \
    SpatialDropout1D, Bidirectional, Conv1D, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import Callback

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

%matplotlib inline

In [ ]:
print(tf.test.gpu_device_name())

/device:GPU:0


In [ ]:
df = pd.read_csv('../data/data.csv')

In [ ]:
df.head()

,word,tag
0,Pengamat,O
1,politik,O
2,dari,O
3,Universitas,B-ORGANIZATION
4,Gadjah,I-ORGANIZATION


In [ ]:
sentences = []
cnt = 1

for i in df.itertuples():
    sentences.append(cnt)
    
    if '.' in str(i.word):
        cnt += 1
        
df['sentence #'] = sentences
df.head()

,word,tag,sentence #
0,Pengamat,O,1
1,politik,O,1
2,dari,O,1
3,Universitas,B-ORGANIZATION,1
4,Gadjah,I-ORGANIZATION,1


In [ ]:
agg_func = lambda s: [(w, t) for w, t in zip(s['word'].values.tolist(), s['tag'].values.tolist())]
grouped = df.groupby('sentence #').apply(agg_func)
sentences = [s for s in grouped]

In [ ]:
words = list(set(df['word'].values))
words.append('PADDING')
num_words = len(words)
tags = list(set(df['tag'].values))
num_tags = len(tags)

In [ ]:
tags = sorted([t for t in tags if t != 'O'], key=lambda x: (x[2], x[0]))
tags.append('O')

In [ ]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [ ]:
tag2idx

{'B-LOCATION': 0,
 'B-ORGANIZATION': 4,
 'B-PERSON': 8,
 'B-QUANTITY': 12,
 'B-TIME': 16,
 'I-LOCATION': 1,
 'I-ORGANIZATION': 5,
 'I-PERSON': 9,
 'I-QUANTITY': 13,
 'I-TIME': 17,
 'L-LOCATION': 2,
 'L-ORGANIZATION': 6,
 'L-PERSON': 10,
 'L-QUANTITY': 14,
 'L-TIME': 18,
 'O': 20,
 'U-LOCATION': 3,
 'U-ORGANIZATION': 7,
 'U-PERSON': 11,
 'U-QUANTITY': 15,
 'U-TIME': 19}

### Pad Sequence

In [ ]:
max_len = 40
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding='post', value=num_words-1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding='post', value=tag2idx['O'])
y = [to_categorical(i, num_classes=num_tags) for i in y]

In [ ]:
x_train, x_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=0.5, random_state=1)

### Pre-trained Word2Vec Embedding

In [ ]:
model = gensim.models.Word2Vec.load("../checkpoint/w2vec_wiki_id_case")

In [ ]:
embedding_matrix = np.zeros((len(words), 400))

for i, w in enumerate(words):
    try:
        embedding_vector = model.wv[w]
        embedding_matrix[i] = embedding_vector
    except:
        embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), 400)

Initial parameters

In [ ]:
def f1_score_micro(y_true, y_pred):
    y_true = np.argmax(y_true.numpy(), axis=-1).reshape(-1)
    y_pred = np.argmax(y_pred.numpy(), axis=-1).reshape(-1)
    
    # label O will be ignored during training and evaluation
    o_pad_idx = np.where(y_true==20) # 20 is the idx for label O
    
    # remove label O
    y_true = np.delete(y_true, o_pad_idx)
    y_pred = np.delete(y_pred, o_pad_idx)
    
    # compute f1 score with micro average
    score = f1_score(y_true, y_pred, average='micro')
    
    return score

In [ ]:
def create_model(x_train, y_train, x_val, y_val, params):
    input_sequence = Input(shape=(max_len, ))
    
    model = Embedding(
        input_dim= embedding_matrix.shape[0], 
        weights=[embedding_matrix], 
        output_dim=embedding_matrix.shape[1], 
        input_length=max_len,
        trainable=False
    )(input_sequence)
    
    # dropout layer
    model = SpatialDropout1D(params['dropout'])(model)
            
    # lstm layer
    model = Bidirectional(LSTM(units=params['lstm_units'], return_sequences=True))(model)
    
    # output layer
    output_sequence = Dense(num_tags, activation='softmax')(model)
    
    # model
    model = Model(input_sequence, output_sequence)
    
    # learning algorithm (optimizer)
    if params['optimizer'] == 'Nadam':
        optm = tf.keras.optimizers.Nadam(lr=params['lr'])
        
    if params['optimizer'] == 'Adam':
        optm = tf.keras.optimizers.Adam(lr=params['lr'])
        
    if params['optimizer'] == 'RMSprop':
        optm = tf.keras.optimizers.RMSprop(lr=params['lr'])
    
    loss = tf.keras.losses.CategoricalCrossentropy()

    model.compile(loss=loss, optimizer=optm, metrics=[f1_score_micro], run_eagerly=True)

    early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_f1_score_micro', mode='max')

    history = model.fit(
      x_train, np.array(y_train),
      validation_data=(x_val, np.array(y_val)),
      epochs=100, verbose=1, callbacks=[early_stopping], batch_size=params['batch_size']
    )

    return history, model

In [ ]:
params = {
    'optimizer': ['Adam'],
    'lr': [0.01],
    'lstm_units': [200],
    'dropout': [0.1, 0.3, 0.5, 0.7],
    'batch_size': [128]
}

In [ ]:
params

{'batch_size': [128],
 'dropout': [0.1, 0.3, 0.5, 0.7],
 'lr': [0.01],
 'lstm_units': [200],
 'optimizer': ['Adam']}

In [ ]:
t = ta.Scan(x=x_train,
    y=np.array(y_train),
    x_val=x_val,
    y_val=np.array(y_val),
    model=create_model,
    params=params,
    experiment_name='bilstm_w2v_opt_v4', 
    val_split=None,
)

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1/100
27/27 [==============================] - 1s 40ms/step - loss: 0.5306 - f1_score_micro: 0.0084 - val_loss: 0.3737 - val_f1_score_micro: 0.0313
Epoch 2/100
27/27 [==============================] - 1s 37ms/step - loss: 0.3243 - f1_score_micro: 0.0792 - val_loss: 0.2814 - val_f1_score_micro: 0.2197
Epoch 3/100
27/27 [==============================] - 1s 39ms/step - loss: 0.2323 - f1_score_micro: 0.3017 - val_loss: 0.2074 - val_f1_score_micro: 0.3838
Epoch 4/100
27/27 [==============================] - 1s 41ms/step - loss: 0.1745 - f1_score_micro: 0.4761 - val_loss: 0.1742 - val_f1_score_micro: 0.5450
Epoch 5/100
27/27 [==============================] - 1s 36ms/step - loss: 0.1412 - f1_score_micro: 0.5858 - val_loss: 0.1473 - val_f1_score_micro: 0.6132
Epoch 6/100
27/27 [==============================] - 1s 40ms/step - loss: 0.1198 - f1_score_micro: 0.6498 - val_loss: 0.1355 - val_f1_score_micro: 0.6145
Epoch 7/100
27/27 [==============================] - 1s 37ms/step - loss: 0.

 25%|██▌       | 1/4 [00:28<01:25, 28.62s/it]

Epoch 1/100
27/27 [==============================] - 1s 39ms/step - loss: 0.5568 - f1_score_micro: 0.0088 - val_loss: 0.3736 - val_f1_score_micro: 0.0392
Epoch 2/100
27/27 [==============================] - 1s 39ms/step - loss: 0.3305 - f1_score_micro: 0.0754 - val_loss: 0.2795 - val_f1_score_micro: 0.1888
Epoch 3/100
27/27 [==============================] - 1s 38ms/step - loss: 0.2374 - f1_score_micro: 0.2851 - val_loss: 0.2018 - val_f1_score_micro: 0.3964
Epoch 4/100
27/27 [==============================] - 1s 41ms/step - loss: 0.1760 - f1_score_micro: 0.4668 - val_loss: 0.1629 - val_f1_score_micro: 0.5659
Epoch 5/100
27/27 [==============================] - 1s 37ms/step - loss: 0.1431 - f1_score_micro: 0.5774 - val_loss: 0.1439 - val_f1_score_micro: 0.5883
Epoch 6/100
27/27 [==============================] - 1s 40ms/step - loss: 0.1206 - f1_score_micro: 0.6462 - val_loss: 0.1307 - val_f1_score_micro: 0.6581
Epoch 7/100
27/27 [==============================] - 1s 37ms/step - loss: 0.

 50%|█████     | 2/4 [00:56<00:56, 28.30s/it]

Epoch 1/100
27/27 [==============================] - 1s 37ms/step - loss: 0.5673 - f1_score_micro: 0.0089 - val_loss: 0.3650 - val_f1_score_micro: 0.0217
Epoch 2/100
27/27 [==============================] - 1s 36ms/step - loss: 0.3283 - f1_score_micro: 0.0849 - val_loss: 0.2705 - val_f1_score_micro: 0.1872
Epoch 3/100
27/27 [==============================] - 1s 41ms/step - loss: 0.2407 - f1_score_micro: 0.2785 - val_loss: 0.1981 - val_f1_score_micro: 0.3866
Epoch 4/100
27/27 [==============================] - 1s 38ms/step - loss: 0.1828 - f1_score_micro: 0.4591 - val_loss: 0.1625 - val_f1_score_micro: 0.5321
Epoch 5/100
27/27 [==============================] - 1s 37ms/step - loss: 0.1515 - f1_score_micro: 0.5409 - val_loss: 0.1409 - val_f1_score_micro: 0.6248
Epoch 6/100
27/27 [==============================] - 1s 38ms/step - loss: 0.1311 - f1_score_micro: 0.6077 - val_loss: 0.1337 - val_f1_score_micro: 0.6380
Epoch 7/100
27/27 [==============================] - 1s 39ms/step - loss: 0.

 75%|███████▌  | 3/4 [01:23<00:28, 28.04s/it]

Epoch 1/100
27/27 [==============================] - 1s 36ms/step - loss: 0.5604 - f1_score_micro: 0.0090 - val_loss: 0.3598 - val_f1_score_micro: 0.0417
Epoch 2/100
27/27 [==============================] - 1s 37ms/step - loss: 0.3249 - f1_score_micro: 0.0871 - val_loss: 0.2584 - val_f1_score_micro: 0.2404
Epoch 3/100
27/27 [==============================] - 1s 41ms/step - loss: 0.2430 - f1_score_micro: 0.2728 - val_loss: 0.2023 - val_f1_score_micro: 0.3837
Epoch 4/100
27/27 [==============================] - 1s 38ms/step - loss: 0.1973 - f1_score_micro: 0.4119 - val_loss: 0.1659 - val_f1_score_micro: 0.5170
Epoch 5/100
27/27 [==============================] - 1s 41ms/step - loss: 0.1689 - f1_score_micro: 0.4974 - val_loss: 0.1472 - val_f1_score_micro: 0.5571
Epoch 6/100
27/27 [==============================] - 1s 39ms/step - loss: 0.1545 - f1_score_micro: 0.5463 - val_loss: 0.1392 - val_f1_score_micro: 0.6448
Epoch 7/100
27/27 [==============================] - 1s 39ms/step - loss: 0.

100%|██████████| 4/4 [02:08<00:00, 32.10s/it]


In [ ]:
# round 3
t.data[['loss', 'f1_score_micro', 'val_loss', 'val_f1_score_micro', 'dropout']] \
    .sort_values(by=['val_f1_score_micro'], ascending=False)

,loss,f1_score_micro,val_loss,val_f1_score_micro,dropout
3,0.047498,0.862486,0.127317,0.746795,0.7
1,0.019924,0.948043,0.138522,0.711894,0.3
2,0.036687,0.898857,0.136642,0.702704,0.5
0,0.015067,0.966158,0.152928,0.679111,0.1


In [ ]:
# round 2
t.data[['loss', 'f1_score_micro', 'val_loss', 'val_f1_score_micro', 'lstm_units']] \
    .sort_values(by=['val_f1_score_micro'], ascending=False)

,loss,f1_score_micro,val_loss,val_f1_score_micro,lstm_units
2,0.002000,0.996915,0.176921,0.721708,200
0,0.020290,0.950416,0.142533,0.711109,50
1,0.016012,0.965461,0.147933,0.693997,100


In [ ]:
# round 1
t.data[['loss', 'f1_score_micro', 'val_loss', 'val_f1_score_micro', 'optimizer', 'lr']] \
    .sort_values(by=['val_f1_score_micro'], ascending=False)

,loss,f1_score_micro,val_loss,val_f1_score_micro,optimizer,lr
1,0.007753,0.984169,0.160842,0.722316,Adam,0.0100
2,0.005736,0.987581,0.180024,0.713748,RMSprop,0.0100
0,0.014240,0.970382,0.140429,0.703763,Nadam,0.0100
4,0.044089,0.887775,0.142158,0.653969,Adam,0.0010
5,0.034772,0.915092,0.146096,0.636344,RMSprop,0.0010
3,0.055261,0.846797,0.141996,0.618761,Nadam,0.0010
8,0.146970,0.535470,0.182531,0.441965,RMSprop,0.0001
7,0.208119,0.325555,0.224032,0.285905,Adam,0.0001
6,0.444643,0.001788,0.433546,0.003381,Nadam,0.0001
